In [1]:
import os
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import nibabel as nib
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.nn import BCEWithLogitsLoss
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold
from collections import Counter
import seaborn as sns
import random
from scipy.ndimage import rotate, zoom
import gc
import warnings
from skimage.filters import threshold_otsu
from scipy import ndimage
from datetime import datetime
warnings.filterwarnings('ignore')

# ---------------------------------------
# CUDA ve CUDNN Ayarları
# ---------------------------------------
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = True
print(f"Using device: {DEVICE}")

# ---------------------------------------
# Eğitim Parametreleri
# ---------------------------------------
num_epochs = 25
batch_size = 8
learning_rate = 1e-4
weight_decay = 1e-4
accumulation_steps = 4
patience = 15
patch_size = (112, 112, 112)  # Updated to match preprocessed data

# Dinamik hiperparametre ayarlama fonksiyonu
def adjust_hyperparameters(fold_results):
    if not fold_results:
        return {
            'learning_rate': learning_rate,
            'batch_size': batch_size,
            'weight_decay': weight_decay,
            'accumulation_steps': accumulation_steps
        }
    
    try:
        last_fold = fold_results[-1]
        val_acc = last_fold['best_val_acc']
        train_loss = last_fold.get('train_losses', [])[-1] if last_fold.get('train_losses') else None
        
        new_params = {
            'learning_rate': learning_rate,
            'batch_size': batch_size,
            'weight_decay': weight_decay,
            'accumulation_steps': accumulation_steps
        }
        
        # Validation accuracy'ye göre learning rate ayarlama
        if val_acc < 0.6:  # Düşük accuracy
            new_params['learning_rate'] *= 1.5  # Learning rate'i artır
        elif val_acc > 0.85:  # Yüksek accuracy
            new_params['learning_rate'] *= 0.8  # Learning rate'i azalt
        
        # Overfitting kontrolü
        if train_loss and last_fold.get('val_losses'):
            train_loss = last_fold['train_losses'][-1]
            val_loss = last_fold['val_losses'][-1]
            if val_loss > train_loss * 1.2:  # Overfitting varsa
                new_params['weight_decay'] *= 1.5  # Regularization'ı artır
                new_params['batch_size'] = min(batch_size * 2, 32)  # Batch size'ı artır
        
        # Batch size sınırlamaları
        new_params['batch_size'] = max(4, min(new_params['batch_size'], 32))
        
        print("\nHiperparametre Ayarlamaları:")
        print(f"Önceki Learning Rate: {learning_rate:.2e} -> Yeni: {new_params['learning_rate']:.2e}")
        print(f"Önceki Batch Size: {batch_size} -> Yeni: {new_params['batch_size']}")
        print(f"Önceki Weight Decay: {weight_decay:.2e} -> Yeni: {new_params['weight_decay']:.2e}")
        
        return new_params
    except Exception as e:
        print(f"Hiperparametre ayarlama hatası: {str(e)}")
        return {
            'learning_rate': learning_rate,
            'batch_size': batch_size,
            'weight_decay': weight_decay,
            'accumulation_steps': accumulation_steps
        }

# ---------------------------------------
# Mixed Precision Training Setup
# ---------------------------------------
scaler = torch.cuda.amp.GradScaler(enabled=True)

# ---------------------------------------
# Data Augmentation Functions
# ---------------------------------------
def random_rotation(img, max_angle=15):
    try:
        angle = random.uniform(-max_angle, max_angle)
        return rotate(img, angle, axes=(0, 1), reshape=False)
    except Exception as e:
        print(f"Rotation hatası: {str(e)}")
        return img

def random_zoom(img, min_zoom=0.9, max_zoom=1.1):
    try:
        zoom_factor = random.uniform(min_zoom, max_zoom)
        return zoom(img, zoom_factor, order=1)
    except Exception as e:
        print(f"Zoom hatası: {str(e)}")
        return img

def random_flip(img, prob=0.5):
    try:
        if random.random() < prob:
            return np.flip(img, axis=0)
        return img
    except Exception as e:
        print(f"Flip hatası: {str(e)}")
        return img

def random_noise(img, mean=0, std=0.01):
    try:
        noise = np.random.normal(mean, std, img.shape)
        return img + noise
    except Exception as e:
        print(f"Noise hatası: {str(e)}")
        return img

# ---------------------------------------
# Dataset ve Model Tanımları
# ---------------------------------------
class ADNI_Dataset(Dataset):
    def __init__(self, paths, labels, transform=None, is_training=True):
        self.paths = paths
        self.labels = labels
        self.transform = transform
        self.is_training = is_training
        self.cache = {}
        self.max_cache_size = 100

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        try:
            if idx in self.cache:
                return self.cache[idx]

            full_path = self.paths[idx]
            img_nib = nib.load(full_path)
            img = img_nib.get_fdata().astype(np.float32)

            # Normalize
            p1, p99 = np.percentile(img, (1, 99))
            img = np.clip(img, p1, p99)
            img = (img - p1) / (p99 - p1 + 1e-6)

            # Brain mask
            thresh = threshold_otsu(img)
            brain_mask = img > thresh
            brain_mask = ndimage.binary_closing(brain_mask, structure=np.ones((3, 3, 3)))
            brain_mask = ndimage.binary_fill_holes(brain_mask)
            brain_mask = ndimage.binary_dilation(brain_mask, structure=np.ones((3, 3, 3)))

            # Largest connected component
            labeled, num_features = ndimage.label(brain_mask)
            if num_features > 1:
                counts = np.bincount(labeled.ravel())
                counts[0] = 0
                largest_label = counts.argmax()
                brain_mask = (labeled == largest_label)

            # Bounding box
            coords = np.array(np.where(brain_mask))
            if len(coords) == 0 or coords.size == 0:
                raise ValueError("Empty coordinate array")
            
            z_min, y_min, x_min = coords.min(axis=1)
            z_max, y_max, x_max = coords.max(axis=1)

            # Center crop
            center_d = (z_min + z_max) // 2
            center_h = (y_min + y_max) // 2
            center_w = (x_min + x_max) // 2

            half = patch_size[0] // 2
            start_d = max(0, center_d - half)
            start_h = max(0, center_h - half)
            start_w = max(0, center_w - half)

            d, h, w = img.shape
            end_d = min(d, start_d + patch_size[0])
            end_h = min(h, start_h + patch_size[0])
            end_w = min(w, start_w + patch_size[0])

            # Adjust sizes
            if end_d - start_d < patch_size[0]:
                start_d = max(0, end_d - patch_size[0])
            if end_h - start_h < patch_size[0]:
                start_h = max(0, end_h - patch_size[0])
            if end_w - start_w < patch_size[0]:
                start_w = max(0, end_w - patch_size[0])

            patch = img[start_d:end_d, start_h:end_h, start_w:end_w]

            # Ensure patch is exactly patch_size
            if patch.shape != patch_size:
                # Create a new array of zeros with the target size
                new_patch = np.zeros(patch_size, dtype=np.float32)
                # Calculate the slices for copying
                d_slice = slice(0, min(patch.shape[0], patch_size[0]))
                h_slice = slice(0, min(patch.shape[1], patch_size[1]))
                w_slice = slice(0, min(patch.shape[2], patch_size[2]))
                # Copy the data
                new_patch[d_slice, h_slice, w_slice] = patch[d_slice, h_slice, w_slice]
                patch = new_patch

            # Z-score normalize
            mean = patch.mean()
            std = patch.std() + 1e-6
            patch = (patch - mean) / std

            # Apply augmentations during training
            if self.is_training:
                patch = random_rotation(patch)
                patch = random_zoom(patch)
                patch = random_flip(patch)
                patch = random_noise(patch)

            # Ensure final size is correct after augmentations
            if patch.shape != patch_size:
                new_patch = np.zeros(patch_size, dtype=np.float32)
                d_slice = slice(0, min(patch.shape[0], patch_size[0]))
                h_slice = slice(0, min(patch.shape[1], patch_size[1]))
                w_slice = slice(0, min(patch.shape[2], patch_size[2]))
                new_patch[d_slice, h_slice, w_slice] = patch[d_slice, h_slice, w_slice]
                patch = new_patch

            img_tensor = torch.from_numpy(patch).float().unsqueeze(0)
            
            if self.transform:
                img_tensor = self.transform(img_tensor)

            label = torch.tensor(self.labels[idx], dtype=torch.long)

            # Cache
            if len(self.cache) >= self.max_cache_size:
                self.cache.pop(next(iter(self.cache)))
            self.cache[idx] = (img_tensor, label)

            return img_tensor, label

        except Exception as e:
            print(f"Error processing {self.paths[idx]}: {str(e)}")
            # Return a zero tensor of the correct size in case of error
            return torch.zeros((1, *patch_size)), torch.tensor(self.labels[idx], dtype=torch.long)

class ImprovedCNN3D(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        
        # Initial convolution with larger kernel
        self.conv1 = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=5, padding=2),
            nn.BatchNorm3d(64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout3d(0.1)
        )
        
        # Residual blocks
        self.res1 = ResidualBlock(64, 128)
        self.res2 = ResidualBlock(128, 256)
        self.res3 = ResidualBlock(256, 512)
        
        # Attention mechanism
        self.attention = SelfAttention(512)
        
        # Global pooling
        self.global_pool = nn.AdaptiveAvgPool3d(1)
        
        # Classifier with more layers and dropout
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
        
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='leaky_relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool3d(x, 2)
        
        x = self.res1(x)
        x = F.max_pool3d(x, 2)
        
        x = self.res2(x)
        x = F.max_pool3d(x, 2)
        
        x = self.res3(x)
        x = F.max_pool3d(x, 2)
        
        x = self.attention(x)
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm3d(out_channels)
        self.conv2 = nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm3d(out_channels)
        
        self.shortcut = nn.Sequential()
        if in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv3d(in_channels, out_channels, kernel_size=1),
                nn.BatchNorm3d(out_channels)
            )
    
    def forward(self, x):
        residual = x
        out = F.leaky_relu(self.bn1(self.conv1(x)), 0.2)
        out = self.bn2(self.conv2(out))
        out += self.shortcut(residual)
        out = F.leaky_relu(out, 0.2)
        return out

class SelfAttention(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.query = nn.Conv3d(in_channels, in_channels//8, 1)
        self.key = nn.Conv3d(in_channels, in_channels//8, 1)
        self.value = nn.Conv3d(in_channels, in_channels, 1)
        self.gamma = nn.Parameter(torch.zeros(1))
    
    def forward(self, x):
        batch_size, C, D, H, W = x.size()
        
        q = self.query(x).view(batch_size, -1, D*H*W)
        k = self.key(x).view(batch_size, -1, D*H*W)
        v = self.value(x).view(batch_size, -1, D*H*W)
        
        attention = F.softmax(torch.bmm(q.permute(0, 2, 1), k), dim=2)
        out = torch.bmm(v, attention.permute(0, 2, 1))
        out = out.view(batch_size, C, D, H, W)
        
        return self.gamma * out + x

# ---------------------------------------
# Veri Yükleme ve Hazırlama
# ---------------------------------------
DATA_DIR = '/kaggle/input/adniprocessed'
classes = {
    'CN_Combined': 0,
    'MCI_Combined': 1,
    'AD_Combined': 2
}

# Veri yükleme ve filtreleme
all_mgz = glob.glob(os.path.join(DATA_DIR, '**', '*.mgz'), recursive=True)
orig_files = [p for p in all_mgz if os.path.basename(p).lower() == 'orig.mgz']

valid_paths, valid_labels = [], []
for orig_path in orig_files:
    group_name = None
    cur_dir = os.path.dirname(orig_path)
    while True:
        cur_dir = os.path.dirname(cur_dir)
        if os.path.basename(cur_dir) in classes:
            group_name = os.path.basename(cur_dir)
            break
        if os.path.abspath(cur_dir) == os.path.abspath(DATA_DIR):
            break

    if group_name is None or os.path.getsize(orig_path) == 0:
        continue

    valid_paths.append(orig_path)
    valid_labels.append(classes[group_name])

# Her sınıftan 300 örnek seç
random.seed(42)
limited_paths, limited_labels = [], []
for cls_name, cls_idx in classes.items():
    cls_indices = [i for i, lab in enumerate(valid_labels) if lab == cls_idx]
    selected = random.sample(cls_indices, min(len(cls_indices), 300))
    for idx in selected:
        limited_paths.append(valid_paths[idx])
        limited_labels.append(cls_idx)

# ---------------------------------------
# K-Fold Cross Validation
# ---------------------------------------
n_splits = 2
kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
all_paths = np.array(valid_paths)
all_labels = np.array(valid_labels)

fold_results = []

for fold in range(n_splits):
    print(f"\nFold {fold+1}/{n_splits}")
    
    # İlk fold'dan sonra hiperparametreleri ayarla
    if fold > 0:
        new_params = adjust_hyperparameters(fold_results)
        learning_rate = new_params['learning_rate']
        batch_size = new_params['batch_size']
        weight_decay = new_params['weight_decay']
        accumulation_steps = new_params['accumulation_steps']
    
    # Data split with stratification
    train_idx, val_idx = list(kfold.split(all_paths, all_labels))[fold]
    paths_train = all_paths[train_idx]
    labels_train = all_labels[train_idx]
    paths_val = all_paths[val_idx]
    labels_val = all_labels[val_idx]
    
    # Print class distribution for this fold
    print("\nTrain set class distribution:")
    print(Counter(labels_train))
    print("\nValidation set class distribution:")
    print(Counter(labels_val))
    
    # Calculate class weights for this fold
    counter_train = Counter(labels_train)
    total_train = sum(counter_train.values())
    class_weights = [np.sqrt(total_train / (len(counter_train) * count)) for count in counter_train.values()]
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(DEVICE)
    print("\nClass weights for this fold:", class_weights)
    
    # Create weighted sampler for training
    sample_weights = [class_weights[label] for label in labels_train]
    train_sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )
    
    # Dataset and DataLoader with weighted sampling
    train_ds = ADNI_Dataset(paths_train, labels_train)
    val_ds = ADNI_Dataset(paths_val, labels_val)
    
    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        sampler=train_sampler,
        num_workers=2,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )
    
    # Model, Loss and Optimizer
    model = ImprovedCNN3D(num_classes=3).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    
    # Cosine annealing with warm restarts
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer,
        T_0=10,
        T_mult=2,
        eta_min=1e-6
    )
    
    # Use weighted loss function with label smoothing
    cls_loss_fn = nn.CrossEntropyLoss(weight=class_weights_tensor, label_smoothing=0.1)
    
    # Training
    best_val_acc = 0.0
    epochs_no_improve = 0
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        total_loss = 0.0
        correct = 0
        total = 0
        
        for i, (imgs, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training")):
            try:
                imgs = imgs.to(DEVICE, non_blocking=True)
                labels = labels.to(DEVICE, non_blocking=True)
                
                optimizer.zero_grad()
                
                with torch.cuda.amp.autocast():
                    outputs = model(imgs)
                    loss = cls_loss_fn(outputs, labels) / accumulation_steps
                
                scaler.scale(loss).backward()
                
                if (i + 1) % accumulation_steps == 0:
                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad()
                
                total_loss += loss.item() * accumulation_steps
                preds = outputs.argmax(dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
                
            except Exception as e:
                print(f"Training batch hatası (epoch {epoch+1}, batch {i}): {str(e)}")
                continue
        
        train_loss = total_loss / len(train_loader)
        train_acc = correct / total
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        
        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for imgs, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
                try:
                    imgs = imgs.to(DEVICE)
                    labels = labels.to(DEVICE)
                    
                    with torch.cuda.amp.autocast():
                        outputs = model(imgs)
                        loss = cls_loss_fn(outputs, labels)
                    
                    val_loss += loss.item()
                    preds = outputs.argmax(dim=1)
                    correct += (preds == labels).sum().item()
                    total += labels.size(0)
                    
                except Exception as e:
                    print(f"Validation batch hatası (epoch {epoch+1}): {str(e)}")
                    continue
        
        val_loss = val_loss / len(val_loader)
        val_acc = correct / total
        val_losses.append(val_loss)
        val_accs.append(val_acc)
        
        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        
        scheduler.step()
        
        if val_acc > best_val_acc + 1e-4:
            best_val_acc = val_acc
            epochs_no_improve = 0
            try:
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'best_val_acc': best_val_acc,
                    'class_weights': class_weights,
                    'fold': fold + 1
                }, f'best_model_fold_{fold+1}.pth')
                print("--Yeni en iyi model kaydedildi--")
            except Exception as e:
                print(f"Model kaydetme hatası: {str(e)}")
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping triggered after {epoch+1} epochs")
                break
        
        # Memory cleanup
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    # Test evaluation for this fold
    try:
        checkpoint = torch.load(f'best_model_fold_{fold+1}.pth')
        model.load_state_dict(checkpoint['model_state_dict'])
        model.eval()
        
        all_targets = []
        all_preds = []
        all_probs = []
        
        with torch.no_grad():
            for imgs, labels in tqdm(val_loader, desc="Testing"):
                try:
                    imgs = imgs.to(DEVICE)
                    labels = labels.to(DEVICE)
                    
                    with torch.cuda.amp.autocast():
                        outputs = model(imgs)
                        probs = F.softmax(outputs, dim=1)
                    
                    preds = outputs.argmax(dim=1)
                    all_targets.extend(labels.cpu().numpy())
                    all_preds.extend(preds.cpu().numpy())
                    all_probs.extend(probs.cpu().numpy())
                    
                except Exception as e:
                    print(f"Test batch hatası: {str(e)}")
                    continue
        
        # Calculate metrics for this fold
        fold_acc = np.mean(np.array(all_preds) == np.array(all_targets))
        fold_report = classification_report(all_targets, all_preds, target_names=["CN", "MCI", "AD"], output_dict=True)
        
        # Save fold results
        fold_results.append({
            'fold': fold + 1,
            'accuracy': fold_acc,
            'classification_report': fold_report,
            'predictions': all_preds,
            'targets': all_targets,
            'probabilities': all_probs,
            'class_weights': class_weights,
            'best_val_acc': best_val_acc,
            'best_epoch': checkpoint['epoch'],
            'train_losses': train_losses,
            'val_losses': val_losses,
            'train_accs': train_accs,
            'val_accs': val_accs
        })
        
        # Save fold results to file
        try:
            with open(f'fold_{fold+1}_results.txt', 'w') as f:
                f.write(f"Fold {fold+1} Results:\n")
                f.write(f"Best Validation Accuracy: {best_val_acc:.4f}\n")
                f.write(f"Best Epoch: {checkpoint['epoch']}\n")
                f.write(f"Final Test Accuracy: {fold_acc:.4f}\n")
                f.write(f"Class Weights Used: {class_weights}\n\n")
                f.write("Classification Report:\n")
                f.write(classification_report(all_targets, all_preds, target_names=["CN", "MCI", "AD"]))
        except Exception as e:
            print(f"Sonuç dosyası yazma hatası: {str(e)}")
        
        # Plot confusion matrix for this fold
        try:
            cm = confusion_matrix(all_targets, all_preds)
            plt.figure(figsize=(8, 6))
            sns.heatmap(cm, annot=True, fmt='d', xticklabels=["CN", "MCI", "AD"], yticklabels=["CN", "MCI", "AD"])
            plt.xlabel("Predicted")
            plt.ylabel("True")
            plt.title(f"Confusion Matrix - Fold {fold+1}")
            plt.savefig(f'confusion_matrix_fold_{fold+1}.png')
            plt.close()
        except Exception as e:
            print(f"Confusion matrix çizim hatası: {str(e)}")
            
    except Exception as e:
        print(f"Test değerlendirme hatası: {str(e)}")
        continue

# Save all fold results to a single file
try:
    with open('all_folds_results.txt', 'w') as f:
        f.write("All Folds Results:\n\n")
        for fold_result in fold_results:
            f.write(f"Fold {fold_result['fold']}:\n")
            f.write(f"Best Validation Accuracy: {fold_result['best_val_acc']:.4f}\n")
            f.write(f"Best Epoch: {fold_result['best_epoch']}\n")
            f.write(f"Final Test Accuracy: {fold_result['accuracy']:.4f}\n")
            f.write(f"Class Weights Used: {fold_result['class_weights']}\n")
            f.write("Classification Report:\n")
            f.write(classification_report(fold_result['targets'], fold_result['predictions'], target_names=["CN", "MCI", "AD"]))
            f.write("\n" + "="*50 + "\n\n")
except Exception as e:
    print(f"Tüm sonuçları kaydetme hatası: {str(e)}")


Using device: cuda



Fold 1/2

Train set class distribution:
Counter({1: 490, 0: 374, 2: 227})

Validation set class distribution:
Counter({1: 491, 0: 374, 2: 226})



Class weights for this fold: [0.861496877967819, 0.9860886199974187, 1.2657234296218451]


Epoch 1/25 - Training:   0%|          | 0/137 [00:00<?, ?it/s]

Epoch 1/25 - Training:   1%|          | 1/137 [01:01<2:19:47, 61.67s/it]

...

Epoch 11/25 - Training:  24%|██▍       | 33/137 [08:51<28:40, 16.54s/it]

Epoch 11/25 - Training:  25%|██▍       | 34/137 [08:52<20:20, 11.85s/it]

Epoch 11/25 - Training:  26%|██▌       | 35/137 [09:16<26:07, 15.37s/it]

Epoch 11/25 - Training:  26%|██▋       | 36/137 [09:17<18:34, 11.03s/it]

Epoch 11/25 - Training:  27%|██▋       | 37/137 [09:40<24:19, 14.60s/it]